# RNA-seq Normalization & QC

이 노트북에서는 RNA-seq count 데이터의 정규화 방법들을 비교하고 적용합니다.

## 왜 정규화가 필요한가?

RNA-seq raw count 데이터는 다음과 같은 기술적 편향(technical bias)을 포함합니다:
- **Library size 차이**: 샘플마다 시퀀싱 깊이가 다름
- **유전자 길이 차이**: 긴 유전자일수록 더 많은 read가 매핑됨
- **RNA composition 차이**: 일부 유전자가 과발현되면 다른 유전자의 상대적 비율이 왜곡됨

정규화를 통해 이러한 기술적 변이를 제거하고, **생물학적 차이만** 비교할 수 있게 됩니다.

## 정규화 방법 비교

| 방법 | 보정 대상 | 목적 | 장점 | 단점 | 사용 상황 |
|------|----------|------|------|------|----------|
| **CPM** (Counts Per Million) | Library size | 샘플 간 비교 | 간단하고 직관적 | 유전자 길이 미보정, composition bias 미보정 | 빠른 탐색, 시각화 |
| **RPKM/FPKM** (Reads/Fragments Per Kilobase Million) | Library size + 유전자 길이 | 샘플 내 유전자 간 비교 | 유전자 길이 보정 | 샘플 간 비교에 부적합, composition bias 미보정 | 단일 샘플 분석 |
| **TPM** (Transcripts Per Million) | Library size + 유전자 길이 | 샘플 간 & 유전자 간 비교 | 샘플 간 비교 가능, 합이 일정(1M) | composition bias 미보정 | 발현량 비교, 시각화 |
| **TMM** (Trimmed Mean of M-values) | Library size + composition | Differential Expression 분석 | composition bias 보정, robust | 유전자 길이 미보정 | **edgeR DE 분석** |
| **DESeq2 (Median of Ratios)** | Library size + composition | Differential Expression 분석 | composition bias 보정, 통계적으로 견고 | 유전자 길이 미보정 | **DESeq2 DE 분석** |

## 각 방법 상세 설명

### 1. CPM (Counts Per Million)
$$CPM = \frac{read\ count}{total\ reads} \times 10^6$$

- **목적**: 시퀀싱 깊이 차이 보정
- **장점**: 계산이 간단, 해석이 쉬움
- **단점**: 유전자 길이와 composition bias를 고려하지 않음
- **언제 사용?**: 빠른 데이터 탐색, 히트맵 시각화

---

### 2. RPKM / FPKM
$$RPKM = \frac{read\ count}{gene\ length\ (kb) \times total\ reads\ (M)}$$

- **목적**: 유전자 길이 + library size 보정
- **장점**: 같은 샘플 내에서 유전자 간 발현량 비교 가능
- **단점**: 샘플 간 합계가 다르므로 샘플 간 직접 비교 부적합
- **FPKM**: Paired-end sequencing에서 fragment 기준으로 계산

---

### 3. TPM (Transcripts Per Million)
$$TPM = \frac{read\ count / gene\ length}{\sum(read\ count / gene\ length)} \times 10^6$$

- **목적**: 샘플 간 비교 가능한 정규화
- **장점**: 모든 샘플에서 합이 1,000,000으로 동일 → 샘플 간 비교 가능
- **단점**: Composition bias 미보정
- **언제 사용?**: 발현량 시각화, 샘플 간 특정 유전자 비교

---

### 4. TMM (edgeR)
- **목적**: Differential Expression 분석을 위한 정규화
- **원리**: M-values (log fold change)의 trimmed mean을 이용해 scaling factor 계산
- **장점**: Composition bias 보정, 극단값에 robust
- **단점**: 유전자 길이 미보정 (DE 분석에서는 불필요)
- **언제 사용?**: edgeR로 DE 분석 시

---

### 5. DESeq2 Normalization (Median of Ratios)
- **목적**: Differential Expression 분석을 위한 정규화
- **원리**: 각 유전자의 geometric mean 대비 ratio의 median으로 size factor 계산
- **장점**: Composition bias 보정, 통계적으로 견고
- **단점**: 유전자 길이 미보정
- **언제 사용?**: DESeq2로 DE 분석 시

## 📌 정규화 방법 선택 가이드

| 분석 목적 | 추천 방법 |
|----------|----------|
| 빠른 데이터 탐색 / 시각화 | CPM, log2(CPM+1) |
| 샘플 간 발현량 비교 | TPM |
| 유전자 간 발현량 비교 (같은 샘플 내) | TPM, RPKM |
| **Differential Expression 분석** | **TMM (edgeR) 또는 DESeq2** |
| Machine Learning 입력 데이터 | TPM 또는 VST (variance stabilizing transformation) |

> ⚠️ **주의**: DE 분석에는 **raw count**를 사용하고, 정규화는 DE 도구 내부에서 처리됩니다. CPM/TPM으로 변환된 데이터를 DE 분석에 사용하면 안 됩니다!

---
## 실습: 정규화 적용

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load count data
counts = pd.read_csv("../data/raw/counts_matrix.tsv", sep="\t", index_col=0)
counts.head()

,GC-MV-11753-1-HepG2-C1_S7,GC-MV-11753-6-HepG2-W3_S12,GC-MV-11753-3-HepG2-C3_S9,GC-MV-11753-5-HepG2-W2_S11,GC-MV-11753-8-HepG2-L2_S14,GC-MV-11753-2-HepG2-C2_S8,GC-MV-11753-7-HepG2-L1_S13,GC-MV-11753-4-HepG2-W1_S10,GC-MV-11753-9-HepG2-L3_S15
geneid,,,,,,,,,
ENSG00000290825.1,0,0,0,0,0,0,0,0,0
ENSG00000223972.6,0,0,0,0,0,0,0,0,0
ENSG00000227232.5,61,65,43,56,52,28,28,14,36
ENSG00000278267.1,0,0,0,0,0,0,0,0,0
ENSG00000243485.5,0,2,0,0,0,0,0,0,0


### CPM 정규화

In [2]:
# CPM (Counts Per Million)
cpm = counts / counts.sum() * 1e6

print("CPM 정규화 후 각 샘플의 합계 (모두 1,000,000):")
print(cpm.sum())

cpm.head()

CPM 정규화 후 각 샘플의 합계 (모두 1,000,000):
GC-MV-11753-1-HepG2-C1_S7     1000000.0
GC-MV-11753-6-HepG2-W3_S12    1000000.0
GC-MV-11753-3-HepG2-C3_S9     1000000.0
GC-MV-11753-5-HepG2-W2_S11    1000000.0
GC-MV-11753-8-HepG2-L2_S14    1000000.0
GC-MV-11753-2-HepG2-C2_S8     1000000.0
GC-MV-11753-7-HepG2-L1_S13    1000000.0
GC-MV-11753-4-HepG2-W1_S10    1000000.0
GC-MV-11753-9-HepG2-L3_S15    1000000.0
dtype: float64


,GC-MV-11753-1-HepG2-C1_S7,GC-MV-11753-6-HepG2-W3_S12,GC-MV-11753-3-HepG2-C3_S9,GC-MV-11753-5-HepG2-W2_S11,GC-MV-11753-8-HepG2-L2_S14,GC-MV-11753-2-HepG2-C2_S8,GC-MV-11753-7-HepG2-L1_S13,GC-MV-11753-4-HepG2-W1_S10,GC-MV-11753-9-HepG2-L3_S15
geneid,,,,,,,,,
ENSG00000290825.1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000223972.6,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000227232.5,2.575096,3.128686,2.17118,2.572592,2.112657,1.398615,1.600925,0.887375,1.596498
ENSG00000278267.1,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000243485.5,0.000000,0.096267,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Log2 CPM (시각화용)

In [ ]:
# Log2 CPM (add small value to avoid log(0))
log_cpm = np.log2(cpm + 1)

# Visualize distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
counts.sum(axis=1).hist(bins=50)
plt.title('Raw Counts Distribution')
plt.xlabel('Total counts per gene')

plt.subplot(1, 2, 2)
log_cpm.mean(axis=1).hist(bins=50)
plt.title('Log2(CPM+1) Distribution')
plt.xlabel('Mean Log2(CPM+1) per gene')

plt.tight_layout()
plt.show()

### TPM 정규화 (유전자 길이 정보 필요)

> 참고: TPM 계산에는 유전자 길이 정보가 필요합니다. 여기서는 예시로 CPM과 비교만 합니다.

In [ ]:
# TPM 계산 예시 (유전자 길이가 있다고 가정)
# gene_lengths = pd.read_csv("gene_lengths.csv", index_col=0)
# rpk = counts.div(gene_lengths['length'] / 1000, axis=0)  # Reads per kilobase
# tpm = rpk / rpk.sum() * 1e6

print("TPM 계산에는 유전자 길이 정보가 필요합니다.")
print("유전자 길이 파일이 있다면 위 코드를 활성화하세요.")

### 정규화 전후 비교

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Before normalization
axes[0].boxplot([counts[col] for col in counts.columns], labels=counts.columns)
axes[0].set_xticklabels(counts.columns, rotation=45, ha='right')
axes[0].set_ylabel('Raw Counts')
axes[0].set_title('Before Normalization (Raw Counts)')
axes[0].set_ylim(0, counts.values.flatten().mean() + 2*counts.values.flatten().std())

# After normalization (Log2 CPM)
axes[1].boxplot([log_cpm[col] for col in log_cpm.columns], labels=log_cpm.columns)
axes[1].set_xticklabels(log_cpm.columns, rotation=45, ha='right')
axes[1].set_ylabel('Log2(CPM + 1)')
axes[1].set_title('After Normalization (Log2 CPM)')

plt.tight_layout()
plt.show()

### 정규화 데이터 저장

In [ ]:
# Save normalized data
cpm.to_csv("../data/processed/counts_cpm.csv")
log_cpm.to_csv("../data/processed/counts_log2cpm.csv")

print("Saved: ../data/processed/counts_cpm.csv")
print("Saved: ../data/processed/counts_log2cpm.csv")